In [1]:


# LangGraph官方文档教程
# https://langchain-ai.github.io/langgraph/tutorials/get-started/6-time-travel/



In [2]:
# Time travel¶  时间旅行 ¶
# In a typical chatbot workflow, the user interacts with the bot one or more times to accomplish a task. Memory and a human-in-the-loop enable checkpoints in the graph state and control future responses.
# 在典型的聊天机器人工作流程中，用户需要与机器人进行一次或多次交互才能完成一项任务。 记忆和人机交互机制会在图形状态中启用检查点，并控制未来的响应。

# What if you want a user to be able to start from a previous response and explore a different outcome? Or what if you want users to be able to rewind your chatbot's work to fix mistakes or try a different strategy, something that is common in applications like autonomous software engineers?
# 如果您希望用户能够从之前的回复开始，探索不同的结果，该怎么办？或者，如果您希望用户能够回放聊天机器人的工作以修复错误或尝试不同的策略（这在自主软件工程师等应用中很常见），该怎么办？

# You can create these types of experiences using LangGraph's built-in time travel functionality.
# 您可以使用 LangGraph 的内置时间旅行功能来创建这些类型的体验。


In [3]:
# Note  笔记

# This tutorial builds on Customize state.
# 本教程以自定义状态为基础。


In [4]:
# 1. Rewind your graph¶
# 1. 回放你的图表 ¶
# Rewind your graph by fetching a checkpoint using the graph's get_state_history method. You can then resume execution at this previous point in time.
# 使用图的 get_state_history 方法获取检查点，即可回溯图。然后，您可以从该时间点恢复执行。


In [5]:
# pip install -U "langchain[google-genai]"


In [6]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [7]:
from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

tool = TavilySearch(max_results=2)
tools = [tool]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [8]:
# 2. Add steps¶
# 2. 添加步骤 ¶
# Add steps to your graph. Every step will be checkpointed in its state history:
# 向图表中添加步骤。每个步骤都会在其状态历史记录中设置检查点：


In [9]:
config = {"configurable": {"thread_id": "1"}}
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "I'm learning LangGraph. "
                    "Could you do some research on it for me?"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (320b91d6-6d93-4ce5-89cd-a6babb81a1a2)
 Call ID: 320b91d6-6d93-4ce5-89cd-a6babb81a1a2
  Args:
    query: LangGraph
    search_depth: advanced
================================= Tool Message =================================
Name: tavily_search

{"query": "LangGraph", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.ibm.com/think/topics/langgraph", "title": "What is LangGraph? - IBM", "content": "LangGraph, created by LangChain, is an open source AI agent framework designed to build, deploy and manage complex generative AI agent workflows. It provides a set of tools and libraries that enable users to create, run and optimize large language models (LLMs) in a scalable and efficient manner. At it

In [10]:
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "Ya that's helpful. Maybe I'll "
                    "build an autonomous agent with it!"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Ya that's helpful. Maybe I'll build an autonomous agent with it!
================================== Ai Message ==================================

That sounds like a great project! Let me know if you have any further questions about LangGraph or need help with anything else as you build your autonomous agent.


In [11]:
# 3. Replay the full state history¶
# 3. 重放完整的状态历史 ¶
# Now that you have added steps to the chatbot, you can replay the full state history to see everything that occurred.
# 现在您已经向聊天机器人添加了步骤，您可以 replay 完整的状态历史记录以查看发生的所有事情。


In [12]:
to_replay = None
for state in graph.get_state_history(config):
    print("Num Messages: ", len(state.values["messages"]), "Next: ", state.next)
    print("-" * 80)
    if len(state.values["messages"]) == 6:
        # We are somewhat arbitrarily selecting a specific state based on the number of chat messages in the state.
        to_replay = state

Num Messages:  6 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  5 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ('__start__',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  3 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  2 Next:  ('tools',)
--------------------------------------------------------------------------------
Num Messages:  1 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  0 Next:  ('__start__',)
--------------------------------------------------------------------------------


In [ ]:
# Checkpoints are saved for every step of the graph. This spans invocations so you can rewind across a full thread's history.
# 检查点会保存图表中每一步的执行情况。这涵盖了所有调用 ，因此您可以回溯整个线程的历史记录。


In [ ]:
# Resume from a checkpoint¶
# 从检查点恢复 ¶
# Resume from the to_replay state, which is after the chatbot node in the second graph invocation. Resuming from this point will call the action node next.
# 从 to_replay 状态恢复，该状态位于第二次图表调用中的 chatbot 机器人节点之后。从此处恢复将调用接下来的操作节点。


In [13]:
print(to_replay.next)
print(to_replay.config)

()
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f060eae-d703-6b6f-8006-98c0fe53cc48'}}


In [14]:
# 4. Load a state from a moment-in-time¶
# 4. 加载某个时刻的状态 ¶
# The checkpoint's to_replay.config contains a checkpoint_id timestamp. Providing this checkpoint_id value tells LangGraph's checkpointer to load the state from that moment in time.
# 检查点的 to_replay.config 文件包含一个 checkpoint_id 时间戳。提供此 checkpoint_id 值会告知 LangGraph 的检查点程序从该时刻开始加载状态。


In [15]:
# The `checkpoint_id` in the `to_replay.config` corresponds to a state we've persisted to our checkpointer.
for event in graph.stream(None, to_replay.config, stream_mode="values"):
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================

That sounds like a great project! Let me know if you have any further questions about LangGraph or need help with anything else as you build your autonomous agent.


In [ ]:

# Congratulations! You've now used time-travel checkpoint traversal in LangGraph. Being able to rewind and explore alternative paths opens up a world of possibilities for debugging, experimentation, and interactive applications.
# 恭喜！ 您现在已经在 LangGraph 中使用了时间旅行检查点遍历。能够回溯并探索其他路径，为调试、实验和交互式应用程序开辟了无限可能。


In [ ]:
# 如何自由地选择要回溯的历史记录并展开对话？

In [ ]:
# 您好！要在 LangGraph 中自由地选择要回溯的历史记录并展开对话，核心在于利用 LangGraph 的**检查点（Checkpointer）**功能。您的 `ChatbotWithTimeTravel.ipynb` 文件已经包含了实现这一功能的基础。

# LangGraph 的 `checkpointer` 会在图的每一步执行后保存当前状态。这些保存的状态被称为“检查点”。通过 `graph.get_state_history()` 方法，您可以获取到一个特定对话线程的所有历史检查点。每个检查点都是一个 `StateSnapshot` 对象，它包含了当时的图状态 (`state.values`) 以及恢复到该状态所需的配置 (`state.config`)。

# ### 实现“时间旅行”的步骤和原理

# 1.  **保存历史记录：**
#     当您使用 `graph.compile(checkpointer=memory)` 编译图时，并为每次 `graph.stream()` 或 `graph.invoke()` 调用提供一个唯一的 `thread_id`（例如 `config = {"configurable": {"thread_id": "1"}}`），LangGraph 就会自动为您保存每次交互的完整图状态历史。

# 2.  **获取历史检查点：**
#     使用 `graph.get_state_history(config)` 方法可以获取到指定 `thread_id` 的所有 `StateSnapshot` 对象。这些 `StateSnapshot` 是按时间倒序排列的（最新的在前面）。

# 3.  **选择回溯点：**
#     您可以遍历这些 `StateSnapshot`，向用户展示每个历史点的摘要信息（例如，当时的消息数量、最后一条消息的内容等），然后让用户选择一个想要回溯的特定检查点。每个 `StateSnapshot` 都有一个 `config` 属性，其中包含一个 `checkpoint_id`，这是该特定历史状态的唯一标识符。

# 4.  **从检查点恢复对话：**
#     一旦用户选择了某个历史检查点，您就可以使用该检查点的 `config` 来调用 `graph.stream()`。当 `graph.stream()` 的第一个参数（即输入消息）为 `None` 时，它会尝试从提供的 `config` 中加载状态并继续执行。如果提供了新的用户输入，它会从该检查点加载状态，然后处理新的输入。

# ### 交互式代码示例

# 以下代码演示了如何在您的 Jupyter Notebook 中实现一个简单的交互式“时间旅行”功能。它会先进行一段对话，然后列出历史记录，让用户选择一个时间点，最后从该时间点继续对话。

# 请确保您已经运行了 `ChatbotWithTimeTravel.ipynb` 中所有之前的代码单元格（特别是设置 API 密钥和构建图的部分），以确保 `graph` 和 `config` 变量可用。

# **新的代码单元格：**


# ### 代码说明：

# 1.  **`run_conversation_and_show_history` 函数：**
#     *   这个函数封装了整个交互流程，接受一个初始用户输入和一个 `thread_id`。
#     *   它首先进行几次对话，以积累一些历史记录。
#     *   然后，它通过 `graph.get_state_history(current_config)` 获取当前线程的所有历史检查点。
#     *   为了方便用户选择，它将历史记录反转为正序（从最旧到最新），并打印出每个历史点的序号、消息数量、最后一条消息的摘要以及下一个执行节点。
#     *   它进入一个循环，提示用户输入想要回溯到的历史点序号，或者选择继续当前对话，或者退出。
#     *   如果用户选择了回溯点，它会使用该历史点的 `StateSnapshot.config` 作为新的配置来调用 `graph.stream(None, selected_snapshot.config, ...)`。这里的 `None` 表示我们不是给图提供新的输入，而是加载并恢复到那个历史状态。
#     *   恢复后，它会打印出该历史点的对话内容，并允许用户输入新的问题，从而在那个历史分支上展开新的对话。
#     *   如果用户选择不回溯，则继续在当前对话的最新状态上进行交互。

# 2.  **`uuid.uuid4()`：**
#     为了确保每次运行此示例时都有一个独立的对话历史，我使用了 `uuid.uuid4()` 来生成一个唯一的 `thread_id`。在实际应用中，`thread_id` 通常与用户会话或聊天室相关联。

# 3.  **`graph.stream(None, to_replay.config, stream_mode="values")`：**
#     这是实现时间旅行的关键。当您将 `stream()` 的第一个参数（输入消息）设置为 `None` 时，LangGraph 会根据传入的 `config`（其中包含了 `checkpoint_id`）从检查点加载完整的历史状态，并从该点开始继续执行。这意味着您可以从历史的任何一个点“复活”对话，并从那里开始新的交互分支。

# ### 如何运行：

# 1.  确保您的 Jupyter Notebook 环境中已安装所有必要的库 (`langgraph`, `langchain-tavily`, `langchain[google-genai]`)。
# 2.  确保您已正确设置 `GOOGLE_API_KEY` 和 `TAVILY_API_KEY` 环境变量（或者通过 `getpass.getpass()` 手动输入）。
# 3.  按顺序运行 `ChatbotWithTimeTravel.ipynb` 中所有之前的代码单元格。
# 4.  将上述提供的完整代码粘贴到一个新的代码单元格中，然后运行它。

# 您将看到一个交互式界面，允许您体验 LangGraph 的“时间旅行”功能。

In [19]:
import uuid # 用于生成唯一的 thread_id

def run_conversation_and_show_history(initial_user_input: str, thread_id: str):
    """
    运行一段对话，然后显示历史记录，并允许用户选择回溯点。
    """
    print(f"\n--- 开始新对话线程: {thread_id} ---")
    current_config = {"configurable": {"thread_id": thread_id}}

    # 第一次交互
    print(f"User: {initial_user_input}")
    events = graph.stream(
        {"messages": [{"role": "user", "content": initial_user_input}]},
        current_config,
        stream_mode="values",
    )
    for event in events:
        if "messages" in event and event["messages"]:
            event["messages"][-1].pretty_print()

    # 第二次交互
    second_input = "Tell me more about its applications."
    print(f"\nUser: {second_input}")
    events = graph.stream(
        {"messages": [{"role": "user", "content": second_input}]},
        current_config,
        stream_mode="values",
    )
    for event in events:
        if "messages" in event and event["messages"]:
            event["messages"][-1].pretty_print()

    # 第三次交互
    third_input = "Can it be used for emotional support chatbots?"
    print(f"\nUser: {third_input}")
    events = graph.stream(
        {"messages": [{"role": "user", "content": third_input}]},
        current_config,
        stream_mode="values",
    )
    for event in events:
        if "messages" in event and event["messages"]:
            event["messages"][-1].pretty_print()

    print("\n--- 对话历史记录 ---")
    history_snapshots = []
    # get_state_history 返回的是倒序，我们把它正序存储方便用户选择
    for i, state in enumerate(reversed(list(graph.get_state_history(current_config)))):
        history_snapshots.append(state)
        # 获取最后一条消息的内容，如果消息列表为空则显示提示
        last_message_content = "No messages at this state"
        if state.values and "messages" in state.values and state.values["messages"]:
            last_message_content = state.values["messages"][-1].content
        
        print(f"[{i}] Checkpoint ID: {state.config['configurable']['checkpoint_id']}")
        print(f"    消息数量: {len(state.values.get('messages', []))}")
        print(f"    最后一条消息: {last_message_content[:80]}...") # 只显示前80个字符
        print(f"    下一个执行节点: {state.next}")
        print("-" * 30)

    while True:
        try:
            choice = input("\n请输入您想回溯到的历史点序号 (例如: 0, 1, 2...)，输入 'n' 继续当前对话，或输入 'q' 退出: ").strip()
            if choice.lower() == 'q':
                print("退出时间旅行模式。")
                return
            elif choice.lower() == 'n':
                print("继续当前对话。")
                selected_snapshot = None # 表示不回溯，继续当前状态
                break
            
            selected_index = int(choice)
            if 0 <= selected_index < len(history_snapshots):
                selected_snapshot = history_snapshots[selected_index]
                print(f"您选择了回溯到历史点 [{selected_index}]。")
                break
            else:
                print("无效的序号，请重新输入。")
        except ValueError:
            print("输入无效，请输入数字或 'n'/'q'。")

    if selected_snapshot:
        # 使用选定的历史检查点的 config 来恢复对话
        print(f"\n--- 从检查点 {selected_snapshot.config['configurable']['checkpoint_id']} 恢复对话 ---")
        
        # 打印恢复后的当前对话内容
        print("\n当前对话内容 (恢复后):")
        if selected_snapshot.values and "messages" in selected_snapshot.values:
            for msg in selected_snapshot.values["messages"]:
                msg.pretty_print()
        else:
            print("无历史消息可显示。")
        
        # 准备一个新的输入，从这个历史点开始新的分支
        while True:
            new_user_input = input("\n从该历史点开始，请输入您的新问题 (输入 'q' 退出): ").strip()
            if new_user_input.lower() == 'q':
                print("结束当前分支对话。")
                break
            
            # 使用选定的 snapshot 的 config 来 stream 新的输入
            new_events = graph.stream(
                {"messages": [{"role": "user", "content": new_user_input}]},
                selected_snapshot.config, # 使用历史检查点的 config
                stream_mode="values",
            )
            for event in new_events:
                if "messages" in event and event["messages"]:
                    event["messages"][-1].pretty_print()
    else:
        # 如果用户选择不回溯，则继续当前对话
        print("\n--- 继续当前对话 ---")
        while True:
            new_user_input = input("\n请输入您的新问题 (输入 'q' 退出): ").strip()
            if new_user_input.lower() == 'q':
                print("结束对话。")
                break
            
            new_events = graph.stream(
                {"messages": [{"role": "user", "content": new_user_input}]},
                current_config, # 使用当前对话的 config
                stream_mode="values",
            )
            for event in new_events:
                if "messages" in event and event["messages"]:
                    event["messages"][-1].pretty_print()

# 运行示例
# 使用一个新的线程ID来避免与之前教程的thread_id冲突
new_thread_id = str(uuid.uuid4())
run_conversation_and_show_history("Hi, I'm Bob. What is LangGraph?", new_thread_id)




--- 开始新对话线程: 86abd04f-e0c7-40a3-8376-9ae7af827eb1 ---
User: Hi, I'm Bob. What is LangGraph?
================================ Human Message =================================

Hi, I'm Bob. What is LangGraph?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (8e60b593-25e6-4b48-90ec-29232bc68a52)
 Call ID: 8e60b593-25e6-4b48-90ec-29232bc68a52
  Args:
    topic: general
    query: What is LangGraph?
================================= Tool Message =================================
Name: tavily_search

{"query": "What is LangGraph?", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.datacamp.com/tutorial/langgraph-tutorial", "title": "LangGraph Tutorial: What Is LangGraph and How to Use It?", "content": "LangGraph is a library within the LangChain ecosystem that provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured and efficient manner.


请输入您想回溯到的历史点序号 (例如: 0, 1, 2...)，输入 'n' 继续当前对话，或输入 'q' 退出:  1


您选择了回溯到历史点 [1]。

--- 从检查点 1f060ed8-9ba6-600e-8000-faeea06c7fb8 恢复对话 ---

当前对话内容 (恢复后):
================================ Human Message =================================

Hi, I'm Bob. What is LangGraph?



从该历史点开始，请输入您的新问题 (输入 'q' 退出):  你记得我的名字吗？


================================ Human Message =================================

你记得我的名字吗？
================================== Ai Message ==================================

是的，你的名字是鲍勃。朗格拉夫是什么？



从该历史点开始，请输入您的新问题 (输入 'q' 退出):  我们之前的对话有哪些内容？


================================ Human Message =================================

我们之前的对话有哪些内容？
================================== Ai Message ==================================

我没有记住之前的对话。



从该历史点开始，请输入您的新问题 (输入 'q' 退出):  q


结束当前分支对话。
